In [1]:
import pandas as pd

### What I want to do:
## a Sankey diagram that breaks down the following:

- total of all funding between 2016 (post-Paris Agreement) to 2021, showing all donors, all recipients, all sectors & objectives 
- how much of the development finance went to climate change (existing csv)
- categorize by funding type
- filter to China
- which sectors the projects serve

### Part 1: Totals

In [2]:
# creating a dataframe for total development finance
# all donors to all recipients for all sectors & objectives during 2016-2021
# Source: https://aid-atlas.org/flows/all/all/all/2016-2021?usdType=usd_commitment

import pandas as pd

data = [
    ['United States', 220], 
    ['Germany', 178], 
    ['japan', 116], 
    ['Turkiye', 33.7], 
    ['EU Institutions, excluding EIB', 114], 
    ['Korea', 45.6], 
    ['Asian Development Bank', 130], 
    ['Development Bank of Latin America', 59], 
    ['International Bank for Reconstruction and Development', 148], 
    ['International Development Association', 158], 
    ['Inter-American Development Bank', 69.9], 
    ['Islamic Development Bank', 50.9], 
    ['International Finance Corporation', 61.4], 
    ['European Bank for Reconstruction and Development', 49.2], 
    ['European Investment Bank', 40.3], 
    ['Others', 91]
]

totals_df = pd.DataFrame(data, columns=['Donor', 'Amount (in billions USD)'])
totals_df.shape

(16, 2)

In [3]:
# this is the total development finance between 2016-2021

total = totals_df['Amount (in billions USD)'].sum()
total

1565.0

### Part 2: Climate change totals (objective)

In [4]:
df = pd.read_csv('climate-change-total.csv')
df.shape

(111028, 37)

In [5]:
df.dtypes

usd_commitment                 int64
usd_disbursement             float64
id                             int64
project_title                 object
short_description             object
long_description              object
donor_name                    object
donor_id                       int64
recipient_name                object
recipient_id                   int64
year_name                      int64
year_id                        int64
lvl_0_sector_name             object
lvl_0_sector_id                int64
lvl_1_sector_name             object
lvl_1_sector_id              float64
lvl_2_sector_name             object
lvl_2_sector_id              float64
grant_type_name               object
grant_type_id                float64
gender                         int64
environment                    int64
trade                          int64
pdgg                           int64
biodiversity                   int64
climate_mitigation             int64
climate_adaptation             int64
d

In [6]:
# to be consistent with the data, we will get only those from 2016-2021

climate_df = df[(df['year_name'] >= 2016) & (df['year_name'] <= 2021)]
climate_df.shape

(70700, 37)

In [7]:
# get the totals

climate_total = climate_df['usd_commitment'].sum() / 1_000_000_000
climate_total

304.06503397

In [8]:
other_purposes_total = total - climate_total
other_purposes_total

1260.93496603

### Part 3: Type of funding

In [9]:
grants_df = climate_df.groupby('grant_type_name')['usd_commitment'].sum().reset_index()
grants_df

# we will highlight ODA grants and ODA loans — official development assistance (bilateral)

,grant_type_name,usd_commitment
0,Equity Investment,1436507614
1,MDB Grants,12353457115
2,MDB Loans,188674626275
3,MDB other,1924927157
4,ODA Grants,40390959084
5,ODA Loans,41626839001
6,Other Official Flows (non Export Credit),3956958512
7,Private Development Finance,3586627747


In [10]:
grants_df = pd.DataFrame(grants_df)
grants_df['in_billions'] = grants_df['usd_commitment'] / 1_000_000_000
grants_df

,grant_type_name,usd_commitment,in_billions
0,Equity Investment,1436507614,1.436508
1,MDB Grants,12353457115,12.353457
2,MDB Loans,188674626275,188.674626
3,MDB other,1924927157,1.924927
4,ODA Grants,40390959084,40.390959
5,ODA Loans,41626839001,41.626839
6,Other Official Flows (non Export Credit),3956958512,3.956959
7,Private Development Finance,3586627747,3.586628


### Part 4: Recipient countries, based on funding

In [11]:
# total ODA grant or loan

oda_df = climate_df[(climate_df['grant_type_name'] == 'ODA Grants') | (climate_df['grant_type_name'] == 'ODA Loans')]
oda_df.shape

(52056, 37)

In [12]:
# this creates a new df showing only recipient countries and total ODA commitments

oda_totals = oda_df.groupby('recipient_name')['usd_commitment'].sum().sort_values(ascending=False).reset_index()
oda_totals

,recipient_name,usd_commitment
0,Developing countries unspecified,12983250551
1,India,10202162704
2,Indonesia,2731909061
3,"South of Sahara, regional",2463056065
4,Mexico,2369339405
...,...,...
164,Democratic People's Republic of Korea,1891075
165,Saint Helena,1505395
166,Montserrat,607841
167,Venezuela,596591


In [13]:
# MDB grant or loan

mdb_df = climate_df[(climate_df['grant_type_name'] == 'MDB Grants') | (climate_df['grant_type_name'] == 'MDB Loans') | (climate_df['grant_type_name'] == 'MDB other')]
mdb_df.shape

(13494, 37)

In [14]:
# this creates a new df showing only recipient countries and total MDB commitments

mdb_totals = mdb_df.groupby('recipient_name')['usd_commitment'].sum().sort_values(ascending=False).reset_index()
mdb_totals

,recipient_name,usd_commitment
0,India,19986561090
1,China (People's Republic of),11335752266
2,Türkiye,9672756722
3,Egypt,8356710580
4,Brazil,8294814087
...,...,...
149,Syrian Arab Republic,1284635
150,Venezuela,556340
151,"Middle Africa, regional",452994
152,Iran,308312


In [15]:
# other funding

other_funds_df = climate_df[(climate_df['grant_type_name'] == 'Private Development Finance') | (climate_df['grant_type_name'] == 'Other Official Flows (non Export Credit)') | (climate_df['grant_type_name'] == 'Equity Investment')]
other_funds_df.shape

(5136, 37)

In [16]:
# this creates a new df showing only recipient countries and total other funding commitments

other_funds_totals = other_funds_df.groupby('recipient_name')['usd_commitment'].sum().sort_values(ascending=False).reset_index()
other_funds_totals

,recipient_name,usd_commitment
0,Developing countries unspecified,2805211344
1,India,1648046145
2,"Africa, regional",671532769
3,"South of Sahara, regional",300237727
4,China (People's Republic of),295640542
...,...,...
103,Guinea,0
104,Jamaica,0
105,Mauritius,0
106,Sri Lanka,0


In [17]:
# add column to break into billions for consistency
oda_totals['in_billions'] = oda_totals['usd_commitment'] / 1_000_000_000

# this is the total ODA commitments to China
oda_china = oda_totals.loc[oda_totals['recipient_name'] == "China (People's Republic of)"]
oda_china

,recipient_name,usd_commitment,in_billions
11,China (People's Republic of),1447875352,1.447875


In [18]:
# add column to break into billions for consistency
mdb_totals['in_billions'] = mdb_totals['usd_commitment'] / 1_000_000_000

# this is the total MDB commitments to China
mdb_china = mdb_totals.loc[mdb_totals['recipient_name'] == "China (People's Republic of)"]
mdb_china

,recipient_name,usd_commitment,in_billions
1,China (People's Republic of),11335752266,11.335752


In [19]:
# add column to break into billions for consistency
other_funds_totals['in_billions'] = other_funds_totals['usd_commitment'] / 1_000_000_000

# this is the total MDB commitments to China
other_funds_china = other_funds_totals.loc[other_funds_totals['recipient_name'] == "China (People's Republic of)"]
other_funds_china

,recipient_name,usd_commitment,in_billions
4,China (People's Republic of),295640542,0.295641


### Part 4: China projects

In [20]:
china_df = df.loc[df['recipient_name'] == "China (People's Republic of)"]
china_df

,usd_commitment,usd_disbursement,id,project_title,short_description,long_description,donor_name,donor_id,recipient_name,recipient_id,...,desertification,rmnch,drr,nutrition,disability,climate_change,mitigation_only,adaptation_only,mitigation_and_adaptation,covid_19
77,389000000,NaN,4499104,HENAN FLOOD EMERGENCY REHABILITATION AND RECOV...,NaN,Henan Flood Emergency Rehabilitation and Recov...,Asian Infrastructure Investment Bank,1024,China (People's Republic of),730,...,0,0,0,0,0,2,0,2,0,0
127,308312389,NaN,4494445,CHINA RE AND STORAGE PROJECT,NaN,China RE and Storage Project,International Bank for Reconstruction and Deve...,901,China (People's Republic of),730,...,0,0,0,0,0,2,2,0,0,0
174,280024913,NaN,4487063,HAJIA RAILWAY,NaN,NaN,International Bank for Reconstruction and Deve...,901,China (People's Republic of),730,...,0,0,0,0,0,2,2,0,0,0
176,278023288,NaN,4488739,HEBEI AIR POLLUTION PREVENTION AND CONTROL PRO...,NaN,NaN,International Bank for Reconstruction and Deve...,901,China (People's Republic of),730,...,0,0,0,0,0,2,2,0,0,0
191,260565406,NaN,4488737,INNOVATIVE FINANCING FOR AIR POLLUTION CONTROL...,NaN,NaN,International Bank for Reconstruction and Deve...,901,China (People's Republic of),730,...,0,0,0,0,0,2,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110989,0,943.0,4472606,Rehabilitation and management strategy for ove...,REHABILITATION AND MANAGEMENT STRATEGY FOR OVE...,In the past 30 years the aquifers in the North...,Switzerland,11,China (People's Republic of),730,...,0,0,0,0,0,2,0,2,0,0
110990,0,1414.0,4472607,Rehabilitation and management strategy for ove...,REHABILITATION AND MANAGEMENT STRATEGY FOR OVE...,In the past 30 years the aquifers in the North...,Switzerland,11,China (People's Republic of),730,...,0,0,0,0,0,2,0,2,0,0
110991,0,7467.0,4472608,Rehabilitation and management strategy for ove...,REHABILITATION AND MANAGEMENT STRATEGY FOR OVE...,In the past 30 years the aquifers in the North...,Switzerland,11,China (People's Republic of),730,...,0,0,0,0,0,2,0,2,0,0
110992,0,2987.0,4472609,Rehabilitation and management strategy for ove...,REHABILITATION AND MANAGEMENT STRATEGY FOR OVE...,In the past 30 years the aquifers in the North...,Switzerland,11,China (People's Republic of),730,...,0,0,0,0,0,2,0,2,0,0


In [21]:
china_df.groupby('lvl_0_sector_name')['usd_commitment'].sum().nlargest(10)

lvl_0_sector_name
Energy                                    5267247990
Transport & Storage                       3681428824
Water Supply & Sanitation                 2550884882
Agriculture, Forestry, Fishing            2440519579
General Environment Protection            2422355734
Other Multi-Sector / Cross-Cutting        1413303358
Banking & Financial Services              1064534492
Industry, Mining, Construction             518333255
Reconstruction Relief & Rehabilitation     411000000
Other Social Infrastructure & Services     177114549
Name: usd_commitment, dtype: int64

In [22]:
china_df.groupby('lvl_1_sector_name')['usd_commitment'].sum().nlargest(10)

lvl_1_sector_name
Forestry                                    819044693
Energy generation, renewable sources        605678619
Agriculture                                 460971600
Energy distribution                         366807096
Energy Policy                               326451809
Industry                                    325734307
Energy generation, non-renewable sources     59789129
Education, Level Unspecified                 56265446
Government & Civil Society-general           34846330
Fishing                                      23997312
Name: usd_commitment, dtype: int64

In [23]:
china_df.groupby('lvl_2_sector_name')['usd_commitment'].sum().nlargest(10)

lvl_2_sector_name
Energy policy and administrative management                     1795900730
Forestry development                                            1337798824
Rail transport                                                  1234516770
Environmental policy and administrative management              1175273496
Waste management/disposal                                       1091697588
Transport policy and administrative management                  1063612499
Urban development and management                                 851241830
Road transport                                                   680075411
Retail gas distribution                                          675222628
Energy generation, renewable sources - multiple technologies     598305467
Name: usd_commitment, dtype: int64

### Part 5: Creating a df for the Sankey

In [24]:
# creating a `df` for the Sankey alluvial diagram
# columns are based on Flourish
# Source: https://helpcenter.flourish.studio/hc/en-us/articles/8761554327183-How-to-format-your-data-to-build-Sankeys-and-alluvial-diagrams

sankey_df = pd.DataFrame(columns=['Source', 'Target', 'Step From', 'Step To', 'Amount'])

In [25]:
sankey_df

,Source,Target,Step From,Step To,Amount


#### a) total development finance --> climate change and other purposes

In [26]:
# climate change totals purposes

step_1 = {
    'Source': 'Total development finance', 
    'Target': 'Climate change', 
    'Step From': 1, 
    'Step To': 2, 
    'Amount': climate_total,
}

step_1_df = pd.DataFrame([step_1], columns=['Source', 'Target', 'Step From', 'Step To', 'Amount'])

sankey_df = pd.concat([sankey_df, step_1_df], ignore_index=True)
sankey_df

/var/folders/4f/fk_hbgy14cg47tw521m3y0xw0000gn/T/ipykernel_52388/3846941015.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sankey_df = pd.concat([sankey_df, step_1_df], ignore_index=True)


,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034


In [27]:
# other purposes

step_1b = {
    'Source': 'Total development finance', 
    'Target': 'Other purposes', 
    'Step From': 1, 
    'Step To': 2, 
    'Amount': other_purposes_total,
}

step_1_df = pd.DataFrame([step_1b], columns=['Source', 'Target', 'Step From', 'Step To', 'Amount'])

sankey_df = pd.concat([sankey_df, step_1_df], ignore_index=True)
sankey_df

,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034
1,Total development finance,Other purposes,1,2,1260.934966


#### b) climate change purpose --> funding

In [28]:
# grants

# rename columns to match sankey_df
grants_df_renamed = grants_df.rename(columns={
    'grant_type_name': 'Target',
    'in_billions': 'Amount'
})

grants_df_renamed

,Target,usd_commitment,Amount
0,Equity Investment,1436507614,1.436508
1,MDB Grants,12353457115,12.353457
2,MDB Loans,188674626275,188.674626
3,MDB other,1924927157,1.924927
4,ODA Grants,40390959084,40.390959
5,ODA Loans,41626839001,41.626839
6,Other Official Flows (non Export Credit),3956958512,3.956959
7,Private Development Finance,3586627747,3.586628


In [29]:
# cleaning up grants_df_renamed

# define grouping
def grant_group(target):
    if 'MDB' in target:
        return 'MDB'
    elif 'ODA' in target:
        return 'ODA'
    else:
        return 'Others'


In [30]:
# this merges the funding groups that we defined

grants_df_renamed['Target'] = grants_df_renamed['Target'].apply(grant_group)
grants_df_renamed = grants_df_renamed.groupby(grants_df_renamed['Target']).agg({
    'usd_commitment': 'sum',
    'Amount': 'sum'
}).reset_index()


In [31]:
source = 'Climate change'
step_from = 2
step_to = 3

temp_list = []
temp_dictionary = {}

for index, row in grants_df_renamed.iterrows():
    temp_dictionary = {
        'Source': source,
        'Target': row['Target'],
        'Step From': step_from,
        'Step To': step_to,
        'Amount': row['Amount']
    }

    temp_list.append(temp_dictionary)

step_2_df = pd.DataFrame(temp_list)
sankey_df = pd.concat([sankey_df, step_2_df], ignore_index=True)
sankey_df

,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034
1,Total development finance,Other purposes,1,2,1260.934966
2,Climate change,MDB,2,3,202.953011
3,Climate change,ODA,2,3,82.017798
4,Climate change,Others,2,3,8.980094


#### c) funding --> China

In [32]:
# this is for ODA funding

temp_list = []
temp_dictionary = {
        'Source': 'ODA',
        'Target': 'China',
        'Step From': 3,
        'Step To': 4,
        'Amount': oda_china['in_billions']
}

step_3_df_oda = pd.DataFrame(temp_dictionary)
sankey_df = pd.concat([sankey_df, step_3_df_oda], ignore_index=True)
sankey_df

,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034
1,Total development finance,Other purposes,1,2,1260.934966
2,Climate change,MDB,2,3,202.953011
3,Climate change,ODA,2,3,82.017798
4,Climate change,Others,2,3,8.980094
5,ODA,China,3,4,1.447875


In [33]:
# this is for MDB funding

temp_dictionary = {
        'Source': 'MDB',
        'Target': 'China',
        'Step From': 3,
        'Step To': 4,
        'Amount': mdb_china['in_billions']
}

step_3_df_mdb = pd.DataFrame(temp_dictionary)
sankey_df = pd.concat([sankey_df, step_3_df_mdb], ignore_index=True)
sankey_df

,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034
1,Total development finance,Other purposes,1,2,1260.934966
2,Climate change,MDB,2,3,202.953011
3,Climate change,ODA,2,3,82.017798
4,Climate change,Others,2,3,8.980094
5,ODA,China,3,4,1.447875
6,MDB,China,3,4,11.335752


In [34]:
# this is for other funding

temp_dictionary = {
        'Source': 'Others',
        'Target': 'China',
        'Step From': 3,
        'Step To': 4,
        'Amount': other_funds_china['in_billions']
}

step_3_df_others = pd.DataFrame(temp_dictionary)
sankey_df = pd.concat([sankey_df, step_3_df_others], ignore_index=True)
sankey_df

,Source,Target,Step From,Step To,Amount
0,Total development finance,Climate change,1,2,304.065034
1,Total development finance,Other purposes,1,2,1260.934966
2,Climate change,MDB,2,3,202.953011
3,Climate change,ODA,2,3,82.017798
4,Climate change,Others,2,3,8.980094
5,ODA,China,3,4,1.447875
6,MDB,China,3,4,11.335752
7,Others,China,3,4,0.295641


In [36]:
sankey_df.to_csv('sankey.csv') # this should be good for data viz

### Other data work: China --> sectors?

In [46]:
proj_sectors = china_df.groupby('lvl_0_sector_name')['usd_commitment'].sum().sort_values(ascending=False)

proj_sectors_df = pd.DataFrame(proj_sectors)
proj_sectors_df['in_billions'] = proj_sectors_df / 1_000_000_000
proj_sectors_df

,usd_commitment,in_billions
lvl_0_sector_name,,
Energy,5267247990,5.267248
Transport & Storage,3681428824,3.681429
Water Supply & Sanitation,2550884882,2.550885
"Agriculture, Forestry, Fishing",2440519579,2.440520
General Environment Protection,2422355734,2.422356
Other Multi-Sector / Cross-Cutting,1413303358,1.413303
Banking & Financial Services,1064534492,1.064534
"Industry, Mining, Construction",518333255,0.518333
Reconstruction Relief & Rehabilitation,411000000,0.411000


#### Maybe chart the top recipient countries per funding type?

In [50]:
oda_totals.head(12) # china is top 11

,recipient_name,usd_commitment,in_billions
0,Developing countries unspecified,12983250551,12.983251
1,India,10202162704,10.202163
2,Indonesia,2731909061,2.731909
3,"South of Sahara, regional",2463056065,2.463056
4,Mexico,2369339405,2.369339
5,Morocco,2004707059,2.004707
6,Viet Nam,1992644751,1.992645
7,"Africa, regional",1846758494,1.846758
8,Brazil,1695936410,1.695936
9,"America, regional",1558717326,1.558717


In [51]:
mdb_totals.head(5) # china is top 2

,recipient_name,usd_commitment,in_billions
0,India,19986561090,19.986561
1,China (People's Republic of),11335752266,11.335752
2,Türkiye,9672756722,9.672757
3,Egypt,8356710580,8.356711
4,Brazil,8294814087,8.294814


In [52]:
other_funds_totals.head(5) # china is top 5

,recipient_name,usd_commitment,in_billions
0,Developing countries unspecified,2805211344,2.805211
1,India,1648046145,1.648046
2,"Africa, regional",671532769,0.671533
3,"South of Sahara, regional",300237727,0.300238
4,China (People's Republic of),295640542,0.295641
